In [163]:
import torch
import torch.nn as nn


In [164]:
class CovertypeClassification(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super().__init__()
    self.layer1 = nn.Linear(54, 128)
    self.act1 = nn.ReLU()
    self.drop1 = nn.Dropout(0.2)
    self.layer2 = nn.Linear(128, 128)
    self.act2 = nn.ReLU()
    self.drop2 = nn.Dropout(0.2)
    self.layer3 = nn.Linear(128, 64)
    self.act3 = nn.ReLU()
    self.output = nn.Linear(64, 7)
    self.softmax = nn.Softmax(dim = 1)
    
   

  def forward(self, x):
    x = self.act1(self.layer1(x))
    x = self.drop1(x)
    x = self.act2(self.layer2(x))
    x = self.drop2(x)
    x = self.act3(self.layer3(x))
    x = self.output(x)
    x = self.softmax(x)
    return x 

In [185]:
from sklearn.metrics import precision_score, recall_score

In [186]:
from sklearn.datasets import fetch_covtype

covertype = fetch_covtype()
X = covertype.data
y = covertype.target


In [187]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

In [188]:
type(X_train)

numpy.ndarray

In [189]:
from torch.utils.data import TensorDataset, DataLoader

X_train_t = torch.tensor(X_train).float()
X_test_t = torch.tensor(X_test).float()
y_train_t = torch.tensor(y_train).long().reshape(-1)
y_train_t = y_train_t -1
y_test_t = torch.tensor(y_test).long().reshape(-1)
y_test_t = y_test_t - 1
unique_labels = y_train_t.unique()
print(unique_labels)



tensor([0, 1, 2, 3, 4, 5, 6])


In [190]:
print(X_train_t.shape)
print(y_train_t.shape)

torch.Size([464809, 54])
torch.Size([464809])


In [191]:
train_dataset = TensorDataset(X_train_t, y_train_t)
test_dataset = TensorDataset(X_test_t, y_test_t)

In [192]:
train_loader = DataLoader(train_dataset, batch_size = 32, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = 32)

In [193]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu") 
print(device)

cpu


In [194]:
net = CovertypeClassification(54, 128, 7).to(device)

In [195]:
import numpy as np

In [196]:
def test_model(net, loss_fn, test_loader):
    total_loss = 0
    net.eval()
    
    with torch.no_grad():
        for x,y in test_loader:
            x = x.to(device) 
            y = y.to(device)
            preds = net(x)
            preds_multiclass = np.argmax(preds, axis = 1)
            loss = loss_fn(preds, y)
            total_loss += loss.item()
            precision = precision_score(y, preds_multiclass, average = 'weighted', zero_division = 0)
            recall = recall_score(y, preds_multiclass, average = 'weighted', zero_division = 0)
            
           
            
    print(total_loss / len(test_loader))  
    print(precision)
    print(recall)
 

In [197]:
optimizer = torch.optim.Adam(net.parameters(), lr = 1e-3, weight_decay = 1e-5)

loss_fn = nn.CrossEntropyLoss()

In [198]:
test_model(net, loss_fn, test_loader)

2.128254163600012
0.0
0.0


In [199]:
EPOCHS = 50

In [200]:
net.train() 
for i in range(EPOCHS):
    if i % 5 == 0:
        print(f"Running epoch: {i}")
    
    
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)
        preds = net(x)
        loss = loss_fn(preds, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Running epoch: 0
Running epoch: 5
Running epoch: 10
Running epoch: 15
Running epoch: 20
Running epoch: 25
Running epoch: 30
Running epoch: 35
Running epoch: 40
Running epoch: 45


In [201]:
test_model(net, loss_fn, test_loader)

1.5255712237258314
0.5969696969696969
0.7272727272727273


In [202]:
best_loss = float('inf')
best_epoch = 0
tolerance = 0
max_tolerance = 2

In [203]:

X_train, X_val, y_train, y_val = train_test_split(X_train_t, y_train_t, test_size=0.2, random_state=42)

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)


train_loader = DataLoader(train_dataset, batch_size = 32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size = 32)


In [204]:

for epoch in range(EPOCHS):
    train_loss = 0.0
    val_loss = 0.0
    net.train()
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)
        preds = net(x)
        loss = loss_fn(preds, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        
    net.eval()
    with torch.no_grad():
        for x_val, y_val in val_loader:
            preds_val = net(x_val)
            loss_val = loss_fn(preds_val, y_val)
            val_loss += loss.item()    
            
 
    train_loss /= len(train_loader)
    val_loss /= len(val_loader)

    print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}")

    
    if val_loss < best_loss:
        best_loss = val_loss
        best_epoch = epoch
        tolerance = 0
    else:
        tolerance += 1

   
    if tolerance > max_tolerance:
        print(f"Early stopping at epoch {epoch+1}")
        break


         
    

Epoch 1: Train Loss=1.5780, Val Loss=1.5940
Epoch 2: Train Loss=1.5842, Val Loss=1.4514
Epoch 3: Train Loss=1.5808, Val Loss=1.8818
Epoch 4: Train Loss=1.5797, Val Loss=1.4608
Epoch 5: Train Loss=1.5763, Val Loss=1.4514
Epoch 6: Train Loss=1.5764, Val Loss=1.1660
Epoch 7: Train Loss=1.5833, Val Loss=1.8762
Epoch 8: Train Loss=1.5771, Val Loss=1.3097
Epoch 9: Train Loss=1.5781, Val Loss=1.4532
Early stopping at epoch 9


In [208]:
learning_rate = [0.001, 0.01, 0.1]
batch_sizes = [32, 64, 128]
size_of_hidden_layers = [64, 128, 256]

best_precision = 0.0
best_model = None

In [209]:
best_loss = float('inf')
best_epoch = 0
tolerance = 0
max_tolerance = 2

In [210]:
EPOCHS = 30


for lr in learning_rate:
    for size in size_of_hidden_layers:
        for batch_size in batch_sizes:
            model = CovertypeClassification(54, size, 7)
            optimizer = torch.optim.Adam(model.parameters(), lr=lr)
            
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size)
            
            for epoch in range(EPOCHS):
                train_loss = 0.0
                val_loss = 0.0
                net.train()
                for x, y in train_loader:
                    x = x.to(device)
                    y = y.to(device)
                    preds = net(x)
                    loss = loss_fn(preds, y)
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    train_loss += loss.item()
        
                net.eval()
                with torch.no_grad():
                    for x_val, y_val in val_loader:
                        preds_val = net(x_val)
                        loss_val = loss_fn(preds_val, y_val)
                        val_loss += loss.item() 

                precision = precision_score(y_val, torch.argmax(preds_val, dim = 1), average = 'weighted', zero_division = 0)
                print(precision)
                
                if precision > best_precision:
                    best_precision = precision
                    best_model = net
                    best_lr = lr
                    best_size = size
                    best_batch_size = batch_size
                    
                train_loss /= len(train_loader)
                val_loss /= len(val_loader)

                print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}")
                
                if val_loss < best_loss:
                    best_loss = val_loss
                    best_epoch = epoch
                    tolerance = 0
                else:
                    tolerance += 1
            
                if tolerance > max_tolerance:
                    print(f"Early stopping at epoch {epoch+1}")
                    best_loss = float('inf')
                    best_epoch = 0
                    tolerance = 0
                    max_tolerance = 2
                    break



print("Best Precision:", best_precision)
print("Best Model:", best_model)   
print("Best learning rate:", best_lr) 
print("Best size of hidden layers:", best_size) 
print("Best batch size:", best_model) 

            

0.25
Epoch 1: Train Loss=1.5740, Val Loss=1.8795
0.25
Epoch 2: Train Loss=1.5747, Val Loss=1.3134
0.25
Epoch 3: Train Loss=1.5740, Val Loss=1.8794
0.25
Epoch 4: Train Loss=1.5737, Val Loss=1.7067
0.25
Epoch 5: Train Loss=1.5741, Val Loss=1.3301
Early stopping at epoch 5
0.5294117647058824
Epoch 1: Train Loss=1.5746, Val Loss=1.5633
0.5294117647058824
Epoch 2: Train Loss=1.5743, Val Loss=1.5647
0.5294117647058824
Epoch 3: Train Loss=1.5747, Val Loss=1.4510
0.5294117647058824
Epoch 4: Train Loss=1.5751, Val Loss=1.5951
0.5294117647058824
Epoch 5: Train Loss=1.5742, Val Loss=1.6464
0.5294117647058824
Epoch 6: Train Loss=1.5736, Val Loss=1.4513
Early stopping at epoch 6
0.5294117647058824
Epoch 1: Train Loss=1.5746, Val Loss=1.5942
0.5294117647058824
Epoch 2: Train Loss=1.5737, Val Loss=1.5945
0.5294117647058824
Epoch 3: Train Loss=1.5741, Val Loss=1.4513
0.5294117647058824
Epoch 4: Train Loss=1.5736, Val Loss=1.5921
0.5294117647058824
Epoch 5: Train Loss=1.5744, Val Loss=1.3085
0.52941176